In [186]:
import numpy as np
import scipy.optimize as opt

In [204]:
rho_tank=2164.87 #kg/m3
m=1000
P=172.37#bar
T=273.15+25
Tref=10+273.15
Po=31.026
tipo='semi' # 'hemi' or 'semi'

In [205]:
import numpy as np
from scipy.optimize import root

# ---------------------------
# Parámetros iniciales y constantes
# ---------------------------
# Puedes ajustar estos valores según tu aplicación.
# En este ejemplo, se definirán algunos valores de ejemplo para P, m y T.
# (Ten en cuenta que en tu código original probablemente ya estén definidos.)
P = P * 100000  # convertir a Pascales.
T = 300         # temperatura en Kelvin

rho_tank=4460   # kg/m3

# Factor y variables usados en los cálculos de volumen (según tu ecuación original)
Z = 0.99704 + P * (6.4149e-9)
V_objetivo = (4157.2 * Z * m * T) / P  # Volumen objetivo calculado (m³)
V=V_objetivo

# Valor inicial para la iteración: x[0] será r y x[1] será L.
x0 = [1.5, 3.0]  # Ejemplo: r=1.5 m y L=3.0 m

# ---------------------------
# Definición del sistema de ecuaciones
# ---------------------------
def ecuaciones(x, V_obj):
    r, L = x[0], x[1]
    # Volumen del cilindro
    if tipo == 'hemi':
        V_tank = (4 / 3) * np.pi * r**3 + (L - 2*r) * np.pi * r**2
    elif tipo == 'semi':
        V_tank = (2 / 3) * np.pi * r**3 + (L - r) * np.pi * r**2
    else:
        raise ValueError("Tipo de tanque no reconocido. Usa 'hemi' o 'semi'.")
    
    # Primera ecuación: el volumen total debe coincidir con V_objetivo.
    f1 = V_tank - V_obj
    # Segunda ecuación: imponer la relación de diseño entre L y r.
    sigma = 15306*100000 #Thedesign strength of carbon fiber laminate inthe hoop direction is assumed to be 15,306bar.
    #ASME Boiler & Pressure Vessel Code, Sección VIII, División 1.
    t = (P * r * 2.25) / (0.8*sigma)
    f2=((r*2)/L)-1.4
    return [f1, f2]

# ---------------------------
# Resolver el sistema usando la función root con el método 'hybr'
# ---------------------------
solucion = root(ecuaciones, x0=x0, args=(V_objetivo,), method='hybr', tol=1e-3)

if solucion.success:
    global L_encontrado
    global V_cilindro_sol
    global V_total_sol
    global mt
    global t
    r, L_encontrado = solucion.x[0], solucion.x[1]
    # Cálculo de volúmenes usando los valores encontrados.
    if tipo == 'hemi':
        V_total_sol = (4 / 3) * np.pi * r**3 + (L_encontrado - 2*r) * np.pi * r**2
    elif tipo == 'semi':
        V_total_sol = (2 / 3) * np.pi * r**3 + (L_encontrado - r) * np.pi * r**2
    else:
        raise ValueError("Tipo de tanque no reconocido. Usa 'hemi' o 'semi'.")

    #print("Volumen objetivo:", V_objetivo, "m3")
    print("Volumen calculado del tanque:", V_total_sol, "m3")
    print("Radio del tanque:", r, "m")
    print("Longitud del tanque:", L_encontrado, "m")

    # ---------------------------
    # Ejemplo de cálculo del espesor (usando una ecuación de pared delgada)
    # ---------------------------
    # Aquí se usa la ecuación: t = (P * r * FS) / (2 * sigma)
    # Por ejemplo, si FS = 1.5 y suponemos sigma = 50e6 Pa (50 MPa):
    sigma = 15306*100000 #Thedesign strength of carbon fiber laminate inthe hoop direction is assumed to be 15,306bar.
    #ASME Boiler & Pressure Vessel Code, Sección VIII, División 1.
    t = (P * r * 1.5) / (0.8*sigma)
    print("Espesor del tanque:", t*1000, "mm")

    # ---------------------------
    # Ejemplo de cálculo de la masa del tanque.
    # Supongamos que la masa se calcula considerando un incremento de r (r + t)
    # y que el volumen se compone de la esfera engrosada y el cilindro engrosado.
    if tipo == 'hemi':
        mt = rho_tank * ( (4/3) * np.pi * (r + t)**3 + np.pi * (r + t)**2 * L_encontrado - V_objetivo )
        mt = mt / 1000  # conversión a toneladas
    elif tipo == 'semi':
        mt = rho_tank * ( (2/3) * np.pi * (r + t)**3 + np.pi * (r + t)**2 * L_encontrado - V_objetivo )
        mt = mt / 1000  # conversión a toneladas
    else:
        raise ValueError("Tipo de tanque no reconocido. Usa 'hemi' o 'semi'.")
    
    print("Masa del tanque:", mt, "ton")
else:
    print("No se encontró solución:", solucion.message)


Volumen calculado del tanque: 80.14009121976606 m3
Radio del tanque: 2.8558180817797267 m
Longitud del tanque: 4.079738686263292 m
Espesor del tanque: 60.30200935373034 mm
Masa del tanque: 360.3165304621548 ton


In [189]:
volumen = (2/3) * np.pi * (17.5**3)
24.3/(volumen/1000000)

2164.8784329141313

In [190]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Parámetros extraídos de la imagen (en cm)
r_internal = 17.5         # Inner Radius (cm)
outer_diameter = 37.4     # Outer Diameter (cm)
r_external = outer_diameter / 2  # Outer Radius (cm)
t = r_external - r_internal      # Espesor de la pared (cm)

# Cálculo de volúmenes (en cm³)
# Volumen de una hemisfera: V = (2/3) * pi * r**3
V_internal = (2/3) * np.pi * (r_internal**3)
V_external = (2/3) * np.pi * (r_external**3)
V_walls = V_external - V_internal
24.3/(V_walls/1000000)



9833.971859795816

In [191]:
P1=Po
P2=P
N_etapas=np.ceil((np.log(P2)/np.log(P1))/np.log(2))
N_etapas

np.float64(7.0)

In [192]:
def Cp_Cv(T):
        return 1.5949974007561318 * T ** (-0.021034016066268504)
def compressor_power(m_dot, T1, P_ratio, gamma, cp, eta):
    """
    Calcula la potencia requerida por un compresor para la compresión adiabática de un gas.

    Parámetros:
      m_dot : float
          Caudal másico del gas (kg/s).
      T1 : float
          Temperatura de entrada al compresor (K).
      P_ratio : float
          Relación de presión (P2 / P1, sin unidades) a la que se desea comprimir.
      gamma : float
          Relación de calores específicos (Cp/Cv).
      cp : float
          Calor específico a presión constante (J/kg·K).
      eta : float
          Eficiencia del compresor (valor entre 0 y 1).

    Retorna:
      power : float
          Potencia del compresor requerida (W, vatios).
    """
    # Cálculo del trabajo específico en J/kg
    specific_work = (cp * T1 * gamma / (gamma - 1)) * ((P_ratio)**((gamma - 1) / gamma) - 1)
    
    # Potencia requerida = caudal másico * trabajo específico, corregido por la eficiencia
    power = m_dot * specific_work / eta
    return power/1000

# Ejemplo de uso:
if __name__ == "__main__":
    # Parámetros de entrada (se pueden ajustar según el caso particular)
    T1      = 300           # Temperatura de entrada en Kelvin
    P_ratio = 10            # Relación de presiones (por ejemplo, de 1 a 10)
    gamma   = 1.40          # Relación de calores específicos (puede variar para hidrógeno, p.ej., ~1.40 en bajas temperaturas)
    cp      = 14300         # Calor específico de hidrógeno a presión constante (J/kg·K). Nota: El valor de hidrógeno es mayor debido a su baja masa molar.
    eta     = 0.75          # Eficiencia del compresor (75%)

    CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
    CP=CP/2*1000
    
    
    power_required = compressor_power(m, T, P_ratio, Cp_Cv(T), CP, eta)
    print(f"La potencia requerida para la compresión es: {power_required:.2f} kW")


La potencia requerida para la compresión es: 18931358.55 kW


In [193]:
HhH2=(19.67099783*(T-Tref) + 0.069681519/2*(T**2-Tref**2) - 0.000200098/3*(T**3-Tref**3) + 2.89493E-07/4*(T**4-Tref*4) - 2.22475E-10/5*(T**5-Tref**5) + 8.81466E-14/6*(T**6-Tref**6) - 1.42043E-17/7*(T**6-Tref**6))/2
HhH2

474.6510488485655

In [194]:
HhH2O = (-22.41701677*(T-Tref) + 0.876972156/2*(T**2-Tref**2) -0.002570393/3*(T**3-Tref**3) + 2.48383E-06/4*(T**4-Tref**4))/18.015
HhH2O

71.38708848243724

In [195]:
rho_tank=4500 #kg/m3
m=1000/3600
P=172.37#bar
R=8.314e-5
T=273.15+25
Tref=10+273.15
Po=31.026

In [196]:
import numpy as np
from scipy.optimize import root

def balance(x, T, Tref, m):
    global e
    global s
    """
    Calcula la función de balance para el proceso,
    devolviendo la diferencia e - s.

    Parámetros:
      x    : vector de incógnitas; se asume que x[0] es la variable a encontrar.
      T    : temperatura en K.
      Tref : temperatura de referencia en K.
      m    : masa del hidrógeno (kg).

    La función utiliza dos series polinómicas (para H2 y H2O)
    para calcular, respectivamente, las energías 'e' y 's', y retorna e - s.
    """
    # Calcular la entalpía (u otra propiedad) para hidrógeno
    Ta=15+273.15
    HhH2 = ( 19.67099783*(T - Tref)
              + (0.069681519/2) * (T**2 - Tref**2)
              - (0.000200098/3) * (T**3 - Tref**3)
              + (2.89493E-07/4) * (T**4 - Tref**4)
              - (2.22475E-10/5) * (T**5 - Tref**5)
              + (8.81466E-14/6) * (T**6 - Tref**6)
              - (1.42043E-17/7) * (T**6 - Tref**6)
            ) / 2
    e = HhH2 * m

    # Calcular la entalpía (u otra propiedad) para agua (H2O)
    HhH2O = ( -22.41701677*(Ta - Tref)
               + (0.876972156/2) * (Ta**2 - Tref**2)
               - (0.002570393/3) * (Ta**3 - Tref**3)
               + (2.48383E-06/4) * (Ta**4 - Tref**4)
             ) / 18.015
    s = HhH2O * x[0]
    
    return e - s


In [197]:
P_diff=P-Po
P_etapa=P_diff/N_etapas

In [198]:
Po=Po
Pf=P
P_acum=[Po]
Temps=[300]
Tref=10+273.15
Qs,Ts,m_H2O,comp,power,Ps=[],[],[],[],[],[]
for i in range(int(N_etapas)):
    P1=sum(P_acum)
    P_acum.append(P_etapa)
    P2=sum(P_acum)
    Z = 0.99704 + P * (6.4149e-9)
    V = ((m)/(2)*8.314*Temps[-1])/sum(P_acum)
    V2=V*(P1/P2)**(1/Cp_Cv(Temps[-1]))
    Z = 0.99704 + P2 * 1e6 * (6.4149e-9)
    Ti=(sum(P_acum)*V2*2)/(m*Z*8.314)
    print('T',Ti)
    # Valor inicial para la incógnita x (en este caso, es un vector de una dimensión)
    x0 = [1.0]

    # Se utiliza scipy.optimize.root con método 'hybr'
    solucion = root(balance, x0, args=(25+273.15, Ti, m), method='hybr', tol=1e-1)

    if solucion.success:
        x_encontrado = solucion.x[0]
        print(f"Solución encontrada: x = {x_encontrado:.6f}")
        Temps.append(25+273.15)
        Qs.append(float(e))
        Ts.append((25+273.15))
        m_H2O.append(float(x_encontrado))
        comp.append('No. '+str(i+1))
        CP=19.67099783 + 0.069681519*T - 0.000200098*T**2 + 2.89493E-07*T**3 - 2.22475E-10*T**4 + 8.81466E-14*T**5 - 1.42043E-17*T**6
        CP=CP/2*1000
        power.append(float(compressor_power(m, Ti, P2/P1, Cp_Cv(Ti), CP, 0.75)))
        Ps.append(P2)
    else:
        print("No se encontró solución:", solucion.message)
        


    

T 158.7904677393928
Solución encontrada: x = 1.051367
T 162.00141174018515
Solución encontrada: x = 1.050879
T 157.7841741067288
Solución encontrada: x = 1.051560
T 151.0880308302572
Solución encontrada: x = 1.053333
T 143.8052473893814
Solución encontrada: x = 1.056202
T 136.6334894918055
Solución encontrada: x = 1.059975
T 129.83677988789555
Solución encontrada: x = 1.064421


In [199]:
import pandas as pd
# Crear un DataFrame con los datos
data = {
	"Compressor": comp,
    "T (K)": Ts,
    "P (bar)": Ps,  # Convertir de Pa a bar
	"Cooling H2O (kg/s)": m_H2O,
	"Q (kW)": Qs,
    "W (kW)": power,
    "Consumption (kW)": np.array(Qs)+ np.array(power)
}

df_compresores = pd.DataFrame(data)
df_compresores.loc[df_compresores.shape[0]] = ["Total", Temps[-1], Ps[-1], sum(m_H2O), sum(Qs), sum(power), sum(Qs)+ sum(power)]

# Mostrar la tabla
df_compresores

,Compressor,T (K),P (bar),Cooling H2O (kg/s),Q (kW),W (kW),Consumption (kW)
0,No. 1,298.15,51.218,1.051367,540.954909,457.935967,998.890875
1,No. 2,298.15,71.410,1.050879,529.022526,301.767031,830.789557
2,No. 3,298.15,91.602,1.051560,544.687203,217.431741,762.118943
3,No. 4,298.15,111.794,1.053333,569.432287,165.303163,734.735451
4,No. 5,298.15,131.986,1.056202,596.159713,130.483501,726.643214
5,No. 6,298.15,152.178,1.059975,622.280770,105.913498,728.194268
6,No. 7,298.15,172.370,1.064421,646.844117,87.851213,734.695330
7,Total,298.15,172.370,7.387737,4049.381525,1466.686112,5516.067638
